<a href="https://colab.research.google.com/github/xiaoxinini53/Train_GPU1/blob/main/Train_GPU1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

# 准备数据集
train_dataset = torchvision.datasets.CIFAR10('../dataset', train=True,
                                             transform=torchvision.transforms.ToTensor(), download=True)
test_dataset = torchvision.datasets.CIFAR10('../dataset', train=False,
                                            transform=torchvision.transforms.ToTensor(), download=True)

train_data_length = len(train_dataset)
test_data_length = len(test_dataset)
print(f'训练集有{train_data_length}张')
print(f'测试集有{test_data_length}张')

# 利用DataLoader来加载数据集
train_loader = DataLoader(train_dataset, batch_size=64)
test_loader = DataLoader(test_dataset, batch_size=64)


# 创建网络模型
class Lms_Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 32, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(1024, 64),
            nn.Linear(64, 10)
        )

    def forward(self, x):
        x = self.model(x)
        return x

Snet = Lms_Net()
if torch.cuda.is_available():
    Snet = Snet.cuda()

# 损失函数
loss_fn = nn.CrossEntropyLoss()
if torch.cuda.is_available():
    loss_fn = loss_fn.cuda()

# 优化器
learning_rate = 1e-2
optimizer = torch.optim.SGD(Snet.parameters(), learning_rate)

# 设置一些网络训练的参数
# 记录训练次数
total_train_step = 0
# 记录测试的次数
total_test_step = 0
# 训练轮数
epoch = 10
# 添加tensorboard
writer = SummaryWriter('../logs')

for i in range(epoch):
    print(f'-----第{i + 1}轮训练-----')

    # 训练步骤开始
    for data in tqdm(train_loader):
        imgs, targets = data
        if torch.cuda.is_available():
            imgs = imgs.cuda()
            targets = targets.cuda()
        outputs = Snet(imgs)
        loss = loss_fn(outputs, targets)

        # 优化器优化模型
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_train_step += 1

        if (total_train_step % 100) == 0:
            writer.add_scalar('train_loss', loss.item(), total_train_step)
    print(f'训练次数: {total_train_step} , loss: {loss.item()}')

    # 测试步骤开始
    total_test_loss = 0
    total_accuracy = 0
    with torch.no_grad():
        for data in test_loader:
            imgs, targets = data
            if torch.cuda.is_available():
                imgs = imgs.cuda()
                targets = targets.cuda()
            outputs = Snet(imgs)
            loss = loss_fn(outputs, targets)
            total_test_loss += loss.item()
            accuracy = (outputs.argmax(1) == targets).sum()
            total_accuracy += accuracy

    print('模型已保存')
    print(f'整体测试集上的loss:{total_test_loss}')
    print(f'整体正确率为:{total_accuracy / test_data_length}')
    torch.save(Snet, f'checkpoint{i + 1}.pth')
    total_test_step += 1
    writer.add_scalar('test_loss', total_test_loss, total_test_step)
    writer.add_scalar('test_accuracy', total_accuracy / test_data_length, total_test_step)

writer.close()


100%|██████████| 170498071/170498071 [00:06<00:00, 27993418.67it/s]


Extracting ../dataset/cifar-10-python.tar.gz to ../dataset
Files already downloaded and verified
训练集有50000张
测试集有10000张
-----第1轮训练-----


100%|██████████| 782/782 [00:16<00:00, 47.79it/s]


训练次数: 782 , loss: 2.138169288635254
模型已保存
整体测试集上的loss:311.59235191345215
整体正确率为:0.2874000072479248
-----第2轮训练-----


100%|██████████| 782/782 [00:09<00:00, 84.19it/s]


...

训练次数: 6256 , loss: 1.7579320669174194
模型已保存
整体测试集上的loss:222.1506451368332
整体正确率为:0.49459999799728394
-----第9轮训练-----


100%|██████████| 782/782 [00:08<00:00, 90.11it/s]


训练次数: 7038 , loss: 1.6485304832458496
模型已保存
整体测试集上的loss:211.7776758670807
整体正确率为:0.5194999575614929
-----第10轮训练-----


100%|██████████| 782/782 [00:09<00:00, 85.35it/s]


训练次数: 7820 , loss: 1.56013023853302
模型已保存
整体测试集上的loss:202.12842667102814
整体正确率为:0.5393999814987183
